In [1]:
import pandas as pd

In [2]:
column_titles_data = ['category', 'primary_image_url', 'all_image_urls', 'attributes', 'index']
column_titles_validation = ['A', 'B']

tsv_data = pd.read_csv("mlchallenge_set_2021.tsv", 
                       sep='\t',
                       lineterminator='\n',
                       names=column_titles_data,
                       header=None
                      )

tsv_data_validation = pd.read_csv("mlchallenge_set_validation.tsv", 
                                  sep='\t',
                                  lineterminator='\n',
                                  names=column_titles_validation,
                                  header=None
                                 )

In [3]:
import re

def parse_attributes(df1):
    attributes = []
    for attrib in df1['attributes']:
        t1 = re.compile(":+").split(attrib[1:-1])
        for i in range(len(t1)):
            t1[i] = t1[i].split(',')
        attr = {}
        for i in range(len(t1)-1):
            if i != len(t1)-2:
                attr[t1[i][-1]] = t1[i+1][:-1]
            else:
                attr[t1[i][-1]] = t1[i+1]
        attributes.append(attr)
    df1['attributes'] = attributes

In [4]:
parse_attributes(tsv_data)

In [5]:
import os

import pickle
import numpy as np
import matplotlib.pyplot as plt
import os
import time
import itertools
from matplotlib import image
import glob as glob
from PIL import Image

import torch
import torchvision
from torchvision import datasets, models, transforms
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchsummary import summary

print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)
# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    print("Using the GPU!")
else:
    print("WARNING: Could not find GPU! Using CPU only. If you want to enable GPU, please to go Edit > Notebook Settings > Hardware Accelerator and select GPU.")


PyTorch Version:  1.3.1
Torchvision Version:  0.4.2


In [6]:
# Download the crap that I need
from os import path
if not path.exists("./images"):
    os.mkdir("images")
    import urllib.request
    img_idx = 0
    for img in tsv_data['primary_image_url']:
        print("Downloading[{1}]: {0}".format(img, img_idx + 1))
        try:
            urllib.request.urlretrieve(img, "./images/" + str(img_idx) + '.JPG')
            if img_idx == 2000:
                break
            img_idx += 1
        except  Exception as e:
            print("Failed: {0}".format(e))

In [10]:
class Edges2Shoes(Dataset):
    def __init__(self, root_dir, split='train', transform=None):
        """
        Args:
            root_dir: the directory of the dataset
            split: "train" or "val"
            transform: pytorch transformations.
        """

        self.transform = transform
        #++++++++++++++++++++++++++++++++++++++++++++++#
        #++++++++++++++++++++++++++++++++++++++++++++++#
        # ============== YOUR CODE HERE ============== #
        # get the the file path to all train/val images
        # Hint: the function glob.glob is useful

        self.files = glob.glob(root_dir + '/' + split + '/*')

        # ============== END OF CODE ================= # 
        #++++++++++++++++++++++++++++++++++++++++++++++#
        #++++++++++++++++++++++++++++++++++++++++++++++#

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        img = Image.open(self.files[idx])
        img = img.resize((128,128))
        img = np.asarray(img)
        if self.transform:
            img = self.transform(img)
        return img

transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
])

#++++++++++++++++++++++++++++++++++++++++++++++#
#++++++++++++++++++++++++++++++++++++++++++++++#
# ============== YOUR CODE HERE ============== #
# Construct the dataloader
# For the train_loader, please use a batch size of 4 and set shuffle to True
# For the test_loader, please use a batch size of 5 and set shuffle to False



tr_dt = Edges2Shoes('./images', 'train', transform)
te_dt = Edges2Shoes('./images', 'val', transform)

train_loader = DataLoader(tr_dt, batch_size=4, shuffle=True)
test_loader = DataLoader(te_dt, batch_size=5, shuffle=False)

# ============== END OF CODE ================= # 
#++++++++++++++++++++++++++++++++++++++++++++++#
#++++++++++++++++++++++++++++++++++++++++++++++#

# Make sure that you have 1,000 training images and 100 testing images before moving on
print('Number of training images {}, number of testing images {}'.format(len(tr_dt), len(te_dt)))

Number of training images 1700, number of testing images 301


In [12]:
#Sample Output used for visualization
test = test_loader.__iter__().__next__()
img_size = 256
fixed_y_ = test[:, :, :, img_size:].cuda()
fixed_x_ = test[:, :, :, 0:img_size].cuda()
print(len(train_loader))
print(len(test_loader))
print(fixed_y_.shape)

# plot sample image
fig, axes = plt.subplots(2, 2)
axes = np.reshape(axes, (4, ))
for i in range(4):
    example = train_loader.__iter__().__next__()[i].numpy().transpose((1, 2, 0))
    mean = np.array([0.5, 0.5, 0.5])
    std = np.array([0.5, 0.5, 0.5])
    example = std * example + mean
    axes[i].imshow(example)
    axes[i].axis('off')
plt.show()

AssertionError: 
Found no NVIDIA driver on your system. Please check that you
have an NVIDIA GPU and installed a driver from
http://www.nvidia.com/Download/index.aspx